In [ ]:
!wget https://bit.ly/fruits_300 -O fruit_300.npy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
fruits=np.load('fruit_300.npy')
fruits.shape

In [ ]:
fruits[0,0,:]

In [ ]:
plt.imshow(fruits[0],cmap='gray')
plt.show()

In [ ]:
plt.imshow(fruits[0],cmap='gray_r')
plt.show()

In [ ]:
fig,axs=plt.subplots(1,2)
# print('fig',fig)
# print('axs',axs)
axs[0].imshow(fruits[200], cmap='gray')
axs[1].imshow(fruits[100], cmap='gray')
plt.show()

In [ ]:
apple=fruits[0:100].reshape(-1,100*100)
pineapple=fruits[100:200].reshape(-1,100*100)
banana=fruits[200:300].reshape(-1,100*100)
apple[0]

In [ ]:
apple.mean(axis=1)

In [ ]:
plt.hist(np.mean(apple,axis=1),alpha=0.7)
plt.hist(np.mean(pineapple,axis=1),alpha=0.7)
plt.hist(np.mean(banana,axis=1),alpha=0.7)
plt.legend(['apple','pineapple','banana'])
plt.show()

In [ ]:
fig,axs=plt.subplots(1,3,figsize=(20,5))
axs[0].bar(range(10000), np.mean(apple,axis=0))
axs[1].bar(range(10000), np.mean(pineapple,axis=0))
axs[2].bar(range(10000), np.mean(banana,axis=0))
plt.show()

In [ ]:
apple_mean=np.mean(apple,axis=0).reshape(100,100)
pineapple_mean=np.mean(pineapple,axis=0).reshape(100,100)
banana_mean=np.mean(banana,axis=0).reshape(100,100)
fig,axs=plt.subplots(1,3,figsize=(20,5))
axs[0].imshow(apple_mean,cmap='gray')
axs[1].imshow(pineapple_mean,cmap='gray')
axs[2].imshow(banana_mean,cmap='gray')
plt.show()

In [ ]:
abs_diff=np.abs(fruits-apple_mean)
abs_mean=np.mean(abs_diff, axis=(1,2))
abs_mean

In [ ]:
apple_index=np.argsort(abs_mean)[:100]
fig,axs=plt.subplots(10,10,figsize=(10,10))
for row in range(10):
  for col in range(10):
    axs[row, col].imshow(fruits[apple_index[row*10+col]], cmap='gray')
    axs[row, col].axis('on')
plt.show()

In [ ]:
abs_diff2=np.abs(fruits-banana_mean)
abs_mean2=np.mean(abs_diff2,axis=(1,2))
banana_index=np.argsort(abs_mean2)[:100]
fig2,axs2=plt.subplots(10,10,figsize=(10,10))
for row in range(10):
  for col in range(10):
    axs2[row, col].imshow(fruits[banana_index[row*10+col]],cmap='gray')
    axs2[row, col].axis('off')
plt.show()

In [ ]:
fruits_2d=fruits.reshape(-1,100*100)
fruits_2d

In [ ]:
from sklearn.cluster import KMeans
km=KMeans(n_clusters=3,random_state=42)
km.fit(fruits_2d)
np.unique(km.labels_,return_counts=True)

In [ ]:
import matplotlib.pyplot as plt
def draw_fruits(arr,ratio=1):
  n=len(arr)
  rows=int(np.ceil(n/10))
  cols=n if rows<2 else 10
  # https://groups.google.com/g/ml-dl-book-qna/c/_xLrD5QLZfo?pli=1
  fig, axs = plt.subplots(rows,cols,figsize=(cols*ratio,rows*ratio),squeeze=False)
  for row in range(rows):
    for col in range(cols):
      if row*10+col < n:
        axs[row,col].imshow(arr[row*10+col],cmap='gray')
      axs[row,col].axis('off')
  plt.show()

In [ ]:
draw_fruits(fruits[km.labels_==0])

In [ ]:
draw_fruits(fruits[km.labels_==1])

In [ ]:
draw_fruits(fruits[km.labels_==2])

In [ ]:
draw_fruits(km.cluster_centers_.reshape(-1,100,100),ratio=3)

In [ ]:
fruits_2d[100:101]

In [ ]:
km.transform(fruits_2d[100:101])

In [ ]:
km.predict(fruits_2d[100:101])

In [ ]:
draw_fruits(fruits[100:101])

In [ ]:
inertia=[]
for k in range(2,7):
  km=KMeans(n_clusters=k,random_state=42)
  km.fit(fruits_2d)
  inertia.append(km.inertia_)
plt.plot(range(2,7),inertia)
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=50)
pca.fit(fruits_2d)
pca.components_

In [ ]:
draw_fruits(pca.components_.reshape(-1,100,100))

In [ ]:
fruits_pca=pca.transform(fruits_2d)
fruits_pca.shape

In [ ]:
fruits_inverse=pca.inverse_transform(fruits_pca)
fruits_reconstruct=fruits_inverse.reshape(-1,100,100)
for start in [0,100,200]:
  draw_fruits(fruits_reconstruct[start:start+100])

In [ ]:
np.sum(pca.explained_variance_ratio_)

In [ ]:
plt.plot(pca.explained_variance_ratio_)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
target=np.array([0]*100+[1]*100+[2]*100)

In [ ]:
from sklearn.model_selection import cross_validate
scores=cross_validate(lr,fruits_2d,target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))

In [ ]:
scores=cross_validate(lr,fruits_pca,target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))

In [ ]:
pca=PCA(n_components=0.5)
pca.fit(fruits_2d)
pca.n_components_

In [ ]:
fruits_pca=pca.transform(fruits_2d)
scores=cross_validate(lr,fruits_pca,target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))

In [ ]:
from sklearn.cluster import KMeans
km=KMeans(n_clusters=3,random_state=42)
km.fit(fruits_pca)
np.unique(km.labels_,return_counts=True)

In [ ]:
for label in range(0,3):
  draw_fruits(fruits[km.labels_==label])

In [ ]:
for label in range(0,3):
  data=fruits_pca[km.labels_ == label]
  plt.scatter(data[:,0],data[:,1])
plt.legend(['pineapple','banana','apple'])
plt.show()

In [ ]:
fruits_pca